<a href="https://colab.research.google.com/github/psg-dsci/nurds/blob/main/Copy_of_Regression_Implementation_FEB22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#requests and beautifulsoup
!pip install requests

In [ ]:
!pip install bs4

In [ ]:
#import the libraries
import requests

In [ ]:
import bs4

In [ ]:
#help(requests)
#help(bs4)

## Location,Rooms,P_type,Status,Area,Price,Bathrooms,Facing
## we have extracted from first page in same process we will go for multiple pages

In [ ]:
base_url = "https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page="
for i in range(1,21):
    url = base_url+str(i)
    print(url)

https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=1
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=2
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=3
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=4
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=5
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=6
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=7
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=8
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=9
https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page=10
https://www.makaan.com/vijayawada-residential-property/buy-

## Multiple pages Data Extraction

In [ ]:
#we need to extract content from every page and store the data in the list
#but if content from one page is extracted (output to be cleared) -->ipython
from IPython.display import clear_output
import time
#create lists to store the data
Locations = [];Area =[];Status=[];Price=[]
Rooms=[];Bathrooms=[];Facing=[];Type=[]
#so now we will follow same pattern which we have done above but for
#multiple listings
import requests,re
from bs4 import BeautifulSoup
#we will use base_url and start extracting listings
Property_Types = ["Apartment","Villa","Builder Floor","Residential Plot",
                  "Independent House","Independent Floor","Studio Apartment"]
#base_url = "https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?page="
for i in range(1,54):
    url = f'https://www.makaan.com/vijayawada-residential-property/buy-property-in-vijayawada-city?propertyType=apartment,builder-floor,villa,residential-plot,independent-house,studio-apartment&page={i}'
    print(f'Scraping Info from Page --> {i}')
    clear_output(wait=True) #this will remove the output
    time.sleep(3)
    #Now we will make connection with each page and extract the contents
    d = requests.get(url)
    soup = BeautifulSoup(d.text, 'html')
    #Now we will go for each listings
    #getting location data
    a = soup.find_all('span',attrs={'itemprop':'addressLocality'})
    for i in a:
        Locations.append(i.text)
    #Number of Rooms
    b = soup.find_all('div',attrs={'class':'title-line'})
    for i in b:
        i = i.span.text
        i = i.replace("Residential Plot","0").replace(" ",'')
        Rooms.append(i)
    #Price data
    c = soup.find_all('div',attrs={'data-type':'price-link'})
    for i in c:
        i = i.text.strip()
        if "Cr" in i:
            i = i.replace(" Cr",'') #we are replacing "Cr" with empty string
            i = float(i)*100 #Converting into Lakhs
        else:
            i = i.replace(" L",'')
            i = float(i)
        Price.append(i)
    #Extracting Area
    d = soup.find_all('td',attrs={'class':'size'})
    for i in d:
        Area.append(i.text.strip())
    #Bathrooms and Facing listings
    e = soup.find_all('ul',attrs={'class':'listing-details'})
    for i in e:
        i = i.text
        bathroom_count = re.findall(r'(\d+) Bathrooms', i)
        if bathroom_count:
          Bathrooms.append(int(bathroom_count[0]))
        else:
          Bathrooms.append('0')
        facing_direction = re.findall(r'(North|South|East|West|NorthEast|NorthWest|SouthEast|SouthWest) facing', i)
        if facing_direction:
          Facing.append(facing_direction[0])
        else:
          Facing.append('None')
    #Status listings
    f = soup.find_all('td',attrs={'class':'val'})
    for i in f:
        Status.append(i.text)
    #Property_Types
    g = soup.find_all('div',attrs={'class':'title-line'})
    for i in g:
        i = i.text
        #print(i)
        for j in Property_Types:
            if j in i: #checking for the property_type in the given list
            #print(j)
                Type.append(j)

Scraping Info from Page --> 53


In [ ]:
print(len(Locations))
print(len(Area))
print(len(Bathrooms))
print(len(Facing))
print(len(Rooms))
print(len(Price))
print(len(Type))
print(len(Status))

1053
1053
1053
1053
1053
1053
1066
1053


In [ ]:
import pandas as pd
#d = pd.DataFrame({'Place':Locations,'Property':Type})
#d #as values are more in Type column we need to avoid it

In [ ]:
#As in above cells we observe Type column has extra entries so we need to make sure
#create a table such that we add Nan values into other cells
#according to values in Type column keep it as max_length value
max_l = 1070 #this value is accordingly value you have received
data = {'Place':Locations,'Area':Area,'Status':Status,
        'Rooms':Rooms,'Bathrooms':Bathrooms,'Facing':Facing,
        'P_Type':Type,'Price':Price}
#from above dictionary we will map according to the max_length of dataframe
for key in data.keys():
    #print(key)
    data[key] += [None] * (max_l- len(data[key]))
df = pd.DataFrame(data)
#df

In [ ]:
df.isnull().sum()

Place        17
Area         17
Status       17
Rooms        17
Bathrooms    17
Facing       17
P_Type        4
Price        17
dtype: int64

In [ ]:
#As we have added missing values in above dataframe -->we can use dropna()
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

Place        0
Area         0
Status       0
Rooms        0
Bathrooms    0
Facing       0
P_Type       0
Price        0
dtype: int64

In [ ]:
#Now we can start building a House Price Prediction Model

In [ ]:
df['Place'].unique()

array(['Poranki', 'Kankipadu', 'Gannavaram', 'Gunadala', 'Gollapudi',
       'Enikepadu', 'Benz Circle', 'Vidhyadharpuram', 'Penamaluru',
       'Payakapuram', 'Guru Nanak Colony', 'Kanuru', 'Nunna',
       'Currency Nagar', 'Edupugallu', 'Ramavarapadu', 'Kesarapalli',
       'Bharathi Nagar', 'Sri Ramachandra Nagar', 'Vijayawada Road',
       'Ramavarapadu Ring', 'Mangalagiri', 'Gudavalli', 'Gosala',
       'Tadigadapa Donka Road', 'Madhuranagar', 'currency nagar',
       'Tadigadapa', 'Gollapudi1', 'Ibrahimpatnam', 'SURAMPALLI',
       'Subba Rao Colony 2nd Cross Road', 'Nidamanuru', 'Kesarapalle',
       'Moghalrajpuram', 'Labbipet', 'Vijayawada Nuzvidu Road',
       'Ajit Singh Nagar', 'AGIRIPALLI', 'Nuzvid To Vijayawada Road',
       'Nuzvid Road', 'Bandar Road', 'Nuzividu', 'Kanchikacherla',
       'Mylavaram', 'G Konduru', 'Punadipadu', 'Jaggayyapet',
       'MylavaramKuntamukkalaVellaturuVijayawada Road',
       'Chennai Vijayawada Highway', 'Guntupalli', 'Tarapet',
       'Tul

In [ ]:
df['Facing'].unique()

array(['West', 'None', 'NorthEast', 'East', 'North', 'South', 'NorthWest',
       'SouthEast', 'SouthWest'], dtype=object)

In [ ]:
df['P_Type'].unique()

array(['Apartment', 'Independent House', 'Independent Floor',
       'Residential Plot', 'Villa', 'Studio Apartment'], dtype=object)

In [ ]:
df['Status'].unique()

array(['Ready to move', 'Under Construction', 'Resale', 'New'],
      dtype=object)

In [ ]:
#handling categorical features ->Label Encoding

In [ ]:
#Before performing LabelEncoding for above features let's take a simple example
a = ['Prashant','Sai','Codegnan',ML']
b = [23,28,5,25]
#combine two lists to create a new collection (dict) -->zip
c = dict(zip(a,b))
c

{'Saketh': 29, 'Sai': 28, 'Codegnan': 5, 'Kits': 25}

In [ ]:
#Now we will apply Label Encoding for our features
from sklearn.preprocessing import LabelEncoder

In [ ]:
label = LabelEncoder()

In [ ]:
#First we will check for Place column
df['Place'] = label.fit_transform(df[['Place']])
#we will showcase how the values are transformed stepbystep
encoded_to_original = dict(zip(label.transform(label.classes_),label.classes_))
encoded_to_original

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{0: 'AGIRIPALLI',
 1: 'Ajit Singh Nagar',
 2: 'Andhra Prabha Colony Road',
 3: 'Ashok Nagar',
 4: 'Auto Nagar',
 5: 'Ayyappa Nagar',
 6: 'Bandar Road',
 7: 'Benz Circle',
 8: 'Bharathi Nagar',
 9: 'Bhavanipuram',
 10: 'Chennai Vijayawada Highway',
 11: 'Currency Nagar',
 12: 'Devi Nagar',
 13: 'Edupugallu',
 14: 'Enikepadu',
 15: 'G Konduru',
 16: 'Gandhi Nagar',
 17: 'Gannavaram',
 18: 'Gollapudi',
 19: 'Gollapudi1',
 20: 'Gosala',
 21: 'Governor Peta',
 22: 'Gudavalli',
 23: 'Gunadala',
 24: 'Guntupalli',
 25: 'Guru Nanak Colony',
 26: 'Ibrahimpatnam',
 27: 'Jaggayyapet',
 28: 'Kanchikacherla',
 29: 'Kandrika',
 30: 'Kanigiri Gurunadham Street',
 31: 'Kankipadu',
 32: 'Kanuru',
 33: 'Kesarapalle',
 34: 'Kesarapalli',
 35: 'LIC Colony',
 36: 'Labbipet',
 37: 'Madhuranagar',
 38: 'Mangalagiri',
 39: 'Milk Factory Road',
 40: 'Moghalrajpuram',
 41: 'Mylavaram',
 42: 'MylavaramKuntamukkalaVellaturuVijayawada Road',
 43: 'Nandigama',
 44: 'Nidamanuru',
 45: 'Nunna',
 46: 'Nuzividu',
 47: 

In [ ]:
df['Status'] = label.fit_transform(df[['Status']])
#we will showcase how the values are transformed stepbystep
encoded_to_original = dict(zip(label.transform(label.classes_),label.classes_))
for encoded,original in encoded_to_original.items():
    print(f'{encoded} - {original}')

0 - New
1 - Ready to move
2 - Resale
3 - Under Construction


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
df['Facing'] = label.fit_transform(df[['Facing']])
#we will showcase how the values are transformed stepbystep
encoded_to_original = dict(zip(label.transform(label.classes_),label.classes_))
for encoded,original in encoded_to_original.items():
    print(f'{encoded} - {original}')

0 - East
1 - None
2 - North
3 - NorthEast
4 - NorthWest
5 - South
6 - SouthEast
7 - SouthWest
8 - West


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
df['P_Type'] = label.fit_transform(df[['P_Type']])
#we will showcase how the values are transformed stepbystep
encoded_to_original = dict(zip(label.transform(label.classes_),label.classes_))
for encoded,original in encoded_to_original.items():
    print(f'{encoded} - {original}')

0 - Apartment
1 - Independent Floor
2 - Independent House
3 - Residential Plot
4 - Studio Apartment
5 - Villa


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053 entries, 0 to 1052
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Place      1053 non-null   int64  
 1   Area       1053 non-null   object 
 2   Status     1053 non-null   int64  
 3   Rooms      1053 non-null   object 
 4   Bathrooms  1053 non-null   object 
 5   Facing     1053 non-null   int64  
 6   P_Type     1053 non-null   int64  
 7   Price      1053 non-null   float64
dtypes: float64(1), int64(4), object(3)
memory usage: 74.0+ KB


In [ ]:
df['Facing'].unique()

array([8, 1, 3, 0, 2, 5, 4, 6, 7])

In [ ]:
#df['Area']
#let us transform Area,Rooms,Bathrooms column values into integer dtype
df['Area'] = df['Area'].astype('int64')

In [ ]:
df['Rooms'] = df['Rooms'].astype('int64')
df['Bathrooms'] = df['Bathrooms'].astype('int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053 entries, 0 to 1052
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Place      1053 non-null   int64  
 1   Area       1053 non-null   int64  
 2   Status     1053 non-null   int64  
 3   Rooms      1053 non-null   int64  
 4   Bathrooms  1053 non-null   int64  
 5   Facing     1053 non-null   int64  
 6   P_Type     1053 non-null   int64  
 7   Price      1053 non-null   float64
dtypes: float64(1), int64(7)
memory usage: 74.0 KB


In [ ]:
#Now let's check for the Outliers -->Visual representation (Boxplot)
import plotly.express as px

In [ ]:
#Boxplot
px.box(df)

In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
#Now we will perform Outlier Detection based on the theoretical knowledge we have gained
def remove_outliers(data,threshold = 3):
    """Function to remove outliers in our dataframe"""
    df_cleaned = data.copy()  #weare creating of our dataframe
    #threshold value determines how far our value lies from first and third quartiles
    for col in df_cleaned.select_dtypes(include = ['int64','float64']).columns:
        #we will first calculate Q1 and Q3 values then IQR
        Q1 = df_cleaned[col].quantile(0.25) #25th percentile
        Q3 = df_cleaned[col].quantile(0.75) #75th percentile
        IQR = Q3-Q1 #calculating IQR
        #now we will lower and upper bounds
        lower_bound = Q1 - threshold*IQR
        upper_bound = Q3 + threshold*IQR
        #Filter the DataFrame to keep only the rows where the
        #column values are within the calculated bounds,
        #effectively removing outliers
        df_cleaned = df_cleaned[
            (df_cleaned[col] >= lower_bound) & (df_cleaned[col] <= upper_bound)]
    return df_cleaned

In [ ]:
#now we will pass dataframe as input to the outliers function (removal of outliers)
new_df = remove_outliers(df)
new_df
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 923 entries, 0 to 1052
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Place      923 non-null    int64  
 1   Area       923 non-null    int64  
 2   Status     923 non-null    int64  
 3   Rooms      923 non-null    int64  
 4   Bathrooms  923 non-null    int64  
 5   Facing     923 non-null    int64  
 6   P_Type     923 non-null    int64  
 7   Price      923 non-null    float64
dtypes: float64(1), int64(7)
memory usage: 64.9 KB


In [ ]:
#Now once Outliers are removed we can go for Modelling first we will apply
#Regression (Linear Regression) and build an end to end Model(Streamlit)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
#let's take a simple example to understand train_test_split
import numpy as np
a,b = np.arange(10).reshape(5,2),range(5)
print(a)
print(b)

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
range(0, 5)


In [ ]:
e,f,g,h = train_test_split(a,b) #here the values willbe consistently modified
print(e)
print(f)
print(g)
print(h)

[[0 1]
 [6 7]
 [8 9]]
[[4 5]
 [2 3]]
[0, 3, 4]
[2, 1]


In [ ]:
e,f,g,h = train_test_split(a,b,random_state=42) #as random_state is used the values will be same
print(e)
print(f)
print(g)
print(h)

[[4 5]
 [0 1]
 [6 7]]
[[2 3]
 [8 9]]
[2, 0, 3]
[1, 4]


In [ ]:
#now based on above concept we will divide our data into dependent and
#independent variables
df.columns

Index(['Place', 'Area', 'Status', 'Rooms', 'Bathrooms', 'Facing', 'P_Type',
       'Price'],
      dtype='object')

In [ ]:
x = new_df.drop('Price',axis=1)
y = new_df['Price']
#y = mx+c

In [ ]:
#we will split the data into train data and test data
#best split 70-30/75-25
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size=0.25,random_state=42)

In [ ]:
#once we have divided the data you can check the samples
len(x);len(x_train)
#x_train
#y_train

692

In [ ]:
#building the model
linear_model = LinearRegression()
linear_model

LinearRegression()

In [ ]:
linear_model.fit(x_train,y_train) #fitting the data

LinearRegression()

In [ ]:
print(dir(linear_model))

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_decision_function', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_parameter_constraints', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_set_intercept', '_validate_data', '_validate_params', 'coef_', 'copy_X', 'feature_names_in_', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'n_features_in_', 'n_jobs', 'positive', 'predict', 'rank_', 'score', 'set_params', 'singular_']


In [ ]:
linear_model.coef_

array([-6.23642074e-02,  1.77329106e-02,  5.54219119e+00,  2.06140709e+01,
       -9.18067338e+00, -1.79465621e-01,  2.10261137e-01])

In [ ]:
linear_model.intercept_ #observe the intercept it should be never negative

-0.024948682137782896

In [ ]:
#now we will start checking the metrics (performance of the model)
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
#check for the predictions and evaluate the metrics
predictions = linear_model.predict(x_test)
#predictions

In [ ]:
print("Mean Squared Error",mean_squared_error(y_test,predictions))
print("R-Squared value",r2_score(y_test,predictions))

Mean Squared Error 277.6632947399053
R-Squared value 0.5566646100500728


In [ ]:
#Now we will check for the predictions giving some test samples
x_test.shape
x_test.columns

Index(['Place', 'Area', 'Status', 'Rooms', 'Bathrooms', 'Facing', 'P_Type'], dtype='object')

In [ ]:
#we can also check the coefficients before going for predictions
lin_m = pd.DataFrame({'Features':x.columns,
                      'Coefficients':linear_model.coef_})
lin_m

,Features,Coefficients
0,Place,-0.062364
1,Area,0.017733
2,Status,5.542191
3,Rooms,20.614071
4,Bathrooms,-9.180673
5,Facing,-0.179466
6,P_Type,0.210261


In [ ]:
#x.info();
print(x_test.head())
print(y_test)

     Place  Area  Status  Rooms  Bathrooms  Facing  P_Type
399     44   960       1      2          2       1       0
974     23  1852       1      3          3       1       0
33      17  1266       1      2          2       0       0
946     28  1332       0      0          0       1       2
367     17  1485       1      3          3       1       1
399    37.00
974    69.50
33     44.13
946     8.88
367    46.00
       ...  
501    35.40
792    29.16
839    27.84
506    58.92
63     76.50
Name: Price, Length: 231, dtype: float64


In [ ]:
#take the inputs from the user to check the predicted price
Rooms = int(input("Enter the number of Rooms"))
Bathrooms = int(input("Enter the number of Bathrooms"))
Status = int(input("Enter the Status"))
Area = int(input("Area:"))
Place = int(input("Enter the location code"))
Facing = int(input("Enter the desired Facing"))
P_Type = int(input("Enter the Property_Type code"))

In [ ]:
#now based on input data we will check the predicted price
price = linear_model.predict(np.array([[Place,Area,Status,Rooms,Bathrooms,Facing,P_Type]]))[0]
price

In [ ]:
#we will create a pickle file for our finally trained model
#pickle,joblib
import pickle

In [ ]:
#first let's create a pickle file for our model
with open("House_Price.pkl",'wb') as f:
    pickle.dump(linear_model,f)

In [ ]:
#we will use our created pickle file for predictions
pred_model = pickle.load(open("House_Price.pkl","rb"))

In [ ]:
#we can use pred_model instance
price = pred_model.predict(np.array([[Place,Area,Status,Rooms,Bathrooms,Facing,P_Type]]))[0]
price

In [ ]:
#No-Free Lunch Theorem --> We cannot say only one model will be best so we apply
#multiple models
#Once our trained model is created as a Pickle file -->we will present to user
#Using Python webservices (Flask,Django) -->Web servers (AWS,Pythonanywhere)
#We can use streamlit (without frontend)

In [ ]:
!pip install streamlit

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_model = PolynomialFeatures(degree=2)

In [ ]:
#Divide the data into train and test data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                test_size=0.3,
                                                random_state=3)

In [ ]:
model = LinearRegression()
#fit the data to the model
model.fit(x_train,y_train)
model.feature_names_in_ #input features
model.n_features_in_

In [ ]:
#let's transform the train_data
x_train = poly_model.fit_transform(x_train)
x_test = poly_model.transform(x_test)

In [ ]:
model = LinearRegression()
#fit the data to the model
model.fit(x_train,y_train)

In [ ]:
poly_model.n_output_features_

In [ ]:
poly_model.get_feature_names_out(x.columns)

In [ ]:
import pandas as pd
poly_coeffs = pd.DataFrame({'Features':poly_model.get_feature_names_out(x.columns),
                           'Coefficients':model.coef_})
poly_coeffs

In [ ]:
#check for predictions and get the metrics
poly_pred = model.predict(x_test)

In [ ]:
print("MSE is",mean_squared_error(y_test,poly_pred))
print("R-Squared value is",r2_score(y_test,poly_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
degrees = [1,2,3,4,5,6,7,8,9,10]
for i in degrees:
    polynomial_features = PolynomialFeatures(degree=i)
    x_poly = polynomial_features.fit_transform(x_train)
    model = LinearRegression()
    # Use cross-validation to assess performance
    scores = cross_val_score(model, x_poly, y_train,
                             scoring='neg_mean_squared_error',
                             cv=5)#5-fold validation
    print(f"Degree {i}: Mean Squared Error: {-np.mean(scores)}")